# Using timeseries forcasting to predict biomarker trends and health outcomes of the monkeys
- Test different time series forcasters (ARIMA, Prophet, or standard catBoost ML) to see if the biomarker and health outcomes of the monkeys can be forecasted on a timeline – very powerful
- If I feel like playing god, I'll also try to forcast death
- It would be great if I can use CatBoost to do the predictions so as to cnsolidate the models in a future iteration, but I'll try all three for accuracy
- I will consider immense success being a model that can forcast a biokarker trend with <25% confidence interval 30 days out, and a model that can forcast a health outcome occuring within 5 days of acctually occuring.
- I will backtest the models with the 20% most recent data
    - and then forcast to the unknown future of the data and validate with the most recent measurments I do not have
    


Article thing with the skforcast: https://www.cienciadedatos.net/documentos/py39-forecasting-time-series-with-skforecast-xgboost-lightgbm-catboost.html

timeseries & catboost examples: https://stackoverflow.com/questions/45262500/example-how-to-use-catboost-with-the-time-series-data

Medium article using catboost for timeseries (no code) https://medium.com/artefact-engineering-and-data-science/time-series-forecasting-how-did-we-put-our-first-sales-forecasting-solution-for-croissants-into-93f9ed45bb49


# Data setup
Unlike the previous predictions, I want to maintain the timeseries information here

# 1.) CatBoost forecasting

# 2.) ARIMA forecasting

# 3.) Prophet forcasting

# Conclusions